## Store raw location records into MongoDB

In [1]:
'''Import and initialize MongoClient'''
from pymongo import MongoClient
con = MongoClient()

We start with storing the raw location records (available in `.txt`) from bus module into MongoDB database. By "raw" location records we mean the location records are as they were stored in Bus Module android application and we have not applied preprocessing to it. Three versions of Bus module application were used for recording location traces with minor changes. In the first version `RawRecords`, the `time` was recorded in <tt>'dd Month YYYY hh mm ss'</tt>  string format (for eg: *8 Jan 2018 07:41:43*). Whereas in the second version `RawRecordEpoch`, the time was recorded in the epoch format. and in the third version `RawRecordEpochSpeed`, the additional parameter `GPS Speed` was recorded. The raw location records correspoding to each version are stored separately in the folder corresponding to version name.

The function `ReadLocationRecordsAndSeparateIntoSegement` reads the raw location records and separates the raw location records into different trip records, if the time between two consecutive records is greater than 30 min. 

Subsequently, it saves the separated trips records into MongoDB with `dd_mm_yyyy__hh_mm_ss.RawRecords` as a collection name. Here <tt>dd_mm_yyyy__hh_mm_ss</tt> represents the start time of the trip and <tt>RawRecords</tt> indicates that the given collection is of raw location record. 

Futher, a status information related to every trip is maintained after every operation in `TripInfo` Collection. This is used at every stages to extract relevant record at each stage of execution.

In [2]:
RouteName='Git_ISCON_PDPU'

If one have executed the current notebook and have created MongoDB database previously then the following code needs to be executed for creating the fresh MongoDB database

In [3]:
'''
Used for deleting the database from MongoDB for clearing the MongoDB database, 
in case one have created the database earlier by executing the below codes.'''
#con.drop_database(RouteName)

'''
In the same way remove the Processed location record with GPS speed
'''
#path = "/".join(os.getcwd().split('/')) + "/LocationRecords/RawRecordEpochSpeedProcessed"
#for file in [f for f in os.listdir(path)]:
#    os.remove(path+"/"+file)

'\nIn the same way remove the Processed location record with GPS speed\n'

In [4]:
import os
import sys

sys.path.append("/".join(os.getcwd().split('/')) +'/Codes/LibCodes')

'''Import project specific library'''
import ReadSeparateTripMongo

path = "/".join(os.getcwd().split('/')) + "/LocationRecords"

'''Read location records folders'''
BusModuleVersion = [f for f in os.listdir(path) if '.md' not in f]

In [5]:
'''For updating the lib changes effects'''
'''
import importlib
importlib.reload(ReadSeparateTripMongo)
'''

'\nimport importlib\nimportlib.reload(ReadSeparateTripMongo)\n'

In [5]:
'''Read location records and separate them into trips. Subsequently store them into MongoDB'''
for RecordType in BusModuleVersion:
    LocationRecordDir = '/'.join([path, RecordType])
    for fileName in [f for f in os.listdir(LocationRecordDir)]:
        
        if RecordType == 'RawRecordEpochSpeed':
            
            ReadSeparateTripMongo.HandlerForNALocation(fileName, 
                                                       LocationRecordDir, 
                                                       LocationRecordDir + 'Processed')
            
            ReadSeparateTripMongo.ReadLocationRecordsAndSeparateIntoSegement(RouteName,
                                                                             fileName,
                                                                             LocationRecordDir + 'Processed',
                                                                             RecordType)

        else:
            ReadSeparateTripMongo.ReadLocationRecordsAndSeparateIntoSegement(RouteName,
                                                                             fileName,
                                                                             LocationRecordDir,
                                                                             RecordType)


Reading file: ISCON_PDPU+1?29_01_2018_16_03_04.txt
Reading file: ISCON_PDPU+1?07_02_2018_09_29_00
Reading file: ISCON_PDPU+1?18_01_2018_07_38_10
Reading file: ISCON_PDPU+1?22_12_2017_07_38_21
Reading file: ISCON_PDPU+1?19_12_2017_18_41_16
Reading file: ISCON_PDPU+1?08_01_2018_07_41_43
Reading file: ISCON_PDPU+1?27_12_2017_07_55_49
Reading file: ISCON_PDPU+1?12_02_2018_08_47_22.txt
Reading file: ISCON_PDPU+1?15_02_2018_16_08_07.txt
Reading file: ISCON_PDPU+1?21_02_2018_16_49_58.txt
Reading file: ISCON_PDPU+1?23_03_2018_08_47_23
Reading file: ISCON_PDPU+1?02_04_18_01_51_00
Reading file: ISCON_PDPU+1?23_03_2018_08_47_22
Reading file: ISCON_PDPU+1?14_02_2018_12_39_44.txt
Reading file: ISCON_PDPU+1?22_02_2018_12_06_55.txt
Reading file: ISCON_PDPU+1?12_02_2018_08_47_22.txt
Reading file: ISCON_PDPU+1?15_02_2018_16_08_07.txt
Reading file: ISCON_PDPU+1?21_02_2018_16_49_58.txt
Reading file: ISCON_PDPU+1?23_03_2018_08_47_23
Reading file: ISCON_PDPU+1?02_04_18_01_51_00
Reading file: ISCON_PDPU+1?2

Note that `ReadSeparateTripMongo.HandlerForNALocation` and `ReadSeparateTripMongo.ReadLocationRecordsAndSeparateIntoSegement` are the project specific library function. One can find help related to project specific functions by executing `FunctionName?`. For instance, on executing the below cell, the help window related to function will pop-up. For further reference, one can also look the library code file in the LibCode directory as mentioned in the file field on executing the below cell.

In [6]:
ReadSeparateTripMongo.ReadLocationRecordsAndSeparateIntoSegement?

###  MongoDB Collection record and it's representation

Let us now look at the `Trip` collection record for one of the trip (let say trip: *29_12_2017__07_37_27*)

In [7]:
[rec for rec in con[RouteName]['TripInfo'].find({'SingleTripInfo':'29_12_2017__07_37_27'})]

[{'_id': ObjectId('5d8b4ac252d4e734fb7eb1f2'),
  'SingleTripInfo': '29_12_2017__07_37_27',
  'filteredLocationRecord': False,
  'DBSCANOp': False,
  'segments': -1,
  'segmentsTimeStamp': []}]

Here, the keys represents the status of the operations applied on the location records. For instance key `filteredLocationRecord` represents whether the given location records are filtered or not, `DBSCANOp` represents whether the DBSCAN based stoppage detection algorithm is applied on the collection records or not, `Segment` represents the number of segments in the location record, after applying interpolation and segmentation procedure. Concretely, the procedure segments the location record if it founds the GPS outage in the location record (the procedure is described later subsection). Likewise, `segmentsTimeStamp` determines the time stamp corresponding to segments in the location records in order to avoid GPS outage from consideration, in all the subsequent procedures. Further, at different stages of modules, the status are computed for all the trips in order to keep the track of operation applied to a given trip. Subsequently, the modules extracts the location record by querying the MongoDB collection *TripInfo* with the *status flag* values. For instance, in order to extract the trips on which filtering is not applied we would query MongoDB as follows:

In [8]:
SingleTripsInfo = [rec['SingleTripInfo'] for rec in con[RouteName]['TripInfo'].find({'filteredLocationRecord': False})]

In [9]:
print(SingleTripsInfo)

['29_01_2018__07_39_47', '30_01_2018__07_42_30', '01_02_2018__07_39_12', '02_02_2018__07_38_50', '18_01_2018__07_38_10', '19_01_2018__07_38_47', '22_01_2018__07_41_04', '22_12_2017__07_38_21', '22_12_2017__18_38_34', '26_12_2017__07_32_35', '19_12_2017__18_41_16', '20_12_2017__07_38_14', '20_12_2017__18_31_19', '21_12_2017__07_52_59', '08_01_2018__07_41_43', '08_01_2018__18_37_49', '09_01_2018__07_40_01', '27_12_2017__07_55_48', '29_12_2017__07_37_27', '01_01_2018__07_38_27', '12_02_2018__07_40_14', '14_02_2018__18_30_22', '15_02_2018__07_45_52', '15_02_2018__16_08_22', '15_02_2018__18_33_19', '16_02_2018__07_45_41', '19_02_2018__07_46_19', '20_02_2018__07_41_48', '20_02_2018__18_31_07', '21_02_2018__07_42_42', '13_03_2018__07_29_52', '14_03_2018__07_35_46', '20_03_2018__07_28_45', '28_03_2018__18_39_21', '21_03_2018__07_32_39', '21_03_2018__18_32_40', '22_03_2018__07_38_43', '14_02_2018__07_41_04', '21_02_2018__18_28_29', '22_02_2018__07_42_45', '12_02_2018__07_40_14', '14_02_2018__18

Let us now look at the fields of collection record for a raw location record

In [10]:
[rec for rec in con[RouteName][SingleTripsInfo[0]+'.RawRecords'].find().limit(1)]

[{'_id': ObjectId('5d8b4abb52d4e734fb7e53b4'),
  'epoch': 1517191787000.0,
  'Longitude': 72.508215,
  'Latitude': 23.03014,
  'Accuracy': 6.599999904632568}]

Here,
`_id` is the unique object id assigned by MongoDB, `Longitude`, `Latitude` corresponds to the location attributes and 
`Accuracy` is the accuracy of location record in meters

## Filtering

The location records are now stored in MongoDB collections, let us know look at the filtering preprocessing steps. 

### Outlier removal
A given location record is considered as an outlier and reomved if

$$ ac > \bar{ac} + 3 \times \sigma_{ac}$$
where $\bar{ac}$ and $\sigma_{ac}$ is the mean and deviation of accuracy considering all the location records of a trip, respectively.

### Segmentation and interpolation
If the consecutive location records are separated by lesser duration ($<15$ seconds) or lesser distance ($<50$ m) then apply interpolation. Else, we separate the location records into different segment and update the information related to segment into the trip status information record of *TripInfo* collection.

Note that we extracted the trips for which filtering is not done using the code 

```python
SingleTripsInfo = [rec['SingleTripInfo'] for rec in con[RouteName]['TripInfo'].find({'filteredLocationRecord': False})]
```

Now, we shall appy filtering into these *SingleTripsInfo*

In [11]:
import Preprocessing

In [12]:
'''For updating the lib changes effects'''
#importlib.reload(Preprocessing)

'For updating the lib changes effects'

In [13]:
for SingleTripInfo in SingleTripsInfo:
    Preprocessing.ApplyFiltering(RouteName,SingleTripInfo)

Executing filtering on 29_01_2018__07_39_47
Executing filtering on 30_01_2018__07_42_30
Executing filtering on 01_02_2018__07_39_12
Executing filtering on 02_02_2018__07_38_50
Executing filtering on 18_01_2018__07_38_10
Executing filtering on 19_01_2018__07_38_47
Executing filtering on 22_01_2018__07_41_04
Executing filtering on 22_12_2017__07_38_21
Executing filtering on 22_12_2017__18_38_34
Executing filtering on 26_12_2017__07_32_35
Executing filtering on 19_12_2017__18_41_16
Executing filtering on 20_12_2017__07_38_14
Executing filtering on 20_12_2017__18_31_19
Executing filtering on 21_12_2017__07_52_59
Executing filtering on 08_01_2018__07_41_43
Executing filtering on 08_01_2018__18_37_49
Executing filtering on 09_01_2018__07_40_01
Executing filtering on 27_12_2017__07_55_48
Executing filtering on 29_12_2017__07_37_27
Executing filtering on 01_01_2018__07_38_27
Executing filtering on 12_02_2018__07_40_14
Executing filtering on 14_02_2018__18_30_22
Executing filtering on 15_02_201

Beside applying the filtering procedure, the `Preprocessing.ApplyFiltering` computes the relative std deviation of the location records in a trip. It also computes the starting hour of the trip. Further, it updates the segmentation information, mean accuracy, std deviation in the accuracy of the location records, trip starting hour in the collection record of a trip in the *TripInfo* collection. For instance, let us now look at the Trip collection record for one of the trip (let say trip: 29_12_2017__07_37_27) as we did earlier.

In [14]:
[rec for rec in con[RouteName]['TripInfo'].find({'SingleTripInfo':'29_12_2017__07_37_27'})]

[{'_id': ObjectId('5d8b4ac252d4e734fb7eb1f2'),
  'SingleTripInfo': '29_12_2017__07_37_27',
  'filteredLocationRecord': True,
  'DBSCANOp': False,
  'segments': 3,
  'segmentsTimeStamp': [[1514513259000.0, 1514513272000.0],
   [1514513291000.0, 1514516035000.0],
   [1514516089000.0, 1514516189000.0]],
  'RelativeSTDAccuracy': 11.342842637811202,
  'TripStartHour': '07',
  'meanAccuracy': 4.157360201823962,
  'stdAccuracy': 0.4715628255798822}]

One may change the trip (i.e. *29_12_2017__07_37_27*) and replace it with any of the other trips of the `SingleTripsInfo` for displaying the corresponding status of the selected trip. 

We would like to draw the attention of readers to the fields of the *TripInfo* collection record. The `Preprocessing.ApplyFiltering` have updated the field `filteredLocationRecord` to **True**, as it has applied the filtering process on a trip. Like wise, the other fields have also updated with the computed values.